# Financial Analysis Agent — Product Demo

This notebook demonstrates how to turn raw financial data into insights and automation with an AI Agent.

**Workflow:** `CSV → Agent (KPIs + LLM) → Report → Automation (Slack/Notion/Make)`

---


## 1) Load sample data

In [ ]:
from src.utils import load_financial_csv
df = load_financial_csv('data/sample.csv')
df.head()  # preview first rows


## 2) Run the Agent (KPIs + insights)

In [ ]:
from src.agents import FinancialAnalysisAgent
agent = FinancialAnalysisAgent()
analysis = agent.generate_report(df)
analysis  # KPIs, AI narrative, recommendations, metrics


## 3) Generate Markdown report

In [ ]:
from src.utils import to_markdown_report, append_metrics_section
import os

report_md = to_markdown_report(analysis['kpis'], analysis['narrative'], analysis['recommendations'])
report_md = append_metrics_section(report_md, analysis.get('metrics', {}))

os.makedirs('outputs', exist_ok=True)
with open('outputs/report.md', 'w', encoding='utf-8') as f:
    f.write(report_md)

print('Report saved to outputs/report.md')
print("\n".join(report_md.splitlines()[:20]))  # preview first lines


## 4) Trigger automation (Webhook / Notion)

In [ ]:
from src.automation import simulate_make_n8n_job
from src.integrations.notion import create_page_markdown

ack = simulate_make_n8n_job(analysis)
print('Webhook response:', ack)

notion_resp = None
import os
if os.environ.get('NOTION_TOKEN') and os.environ.get('NOTION_PARENT_PAGE_ID'):
    notion_resp = create_page_markdown(os.environ['NOTION_PARENT_PAGE_ID'], 'Financial Analysis Report', report_md)
    print('Notion response:', notion_resp)
else:
    print('Notion export skipped.')


---
### Notes
- Real LLM (LangChain + OpenAI/Mistral) can be enabled via `OPENAI_API_KEY`.  
- Automations integrate easily with Slack, Notion, Make or n8n.  
- This notebook shows the **end-to-end product flow**, not classroom exercises.  
